In [ ]:
import numpy as np
import matplotlib.pyplot as plt

world_size_1d = 20

def plot_histogram(bel):
    plt.cla()
    plt.bar(range(0, bel.shape[0]), bel, width=1.0)
    plt.axis([0, bel.shape[0], 0, 1])
    plt.draw()
    plt.pause(0.25)

# Motion models as convolution kernels
def motion_model(bel, u, x):
    # Returns back p(x|u, x_prime)
    # where the index into the kernel is x_prime
    mm_kernel = None

    # Just create fwd motion model kernel and flip it
    # if motion is back
    if u == "back":
        x = world_size_1d - (x+1)

    if x == 0:
        mm_kernel = np.hstack(
            (0.25, np.zeros(world_size_1d - 1)))
    elif x == 1:
        mm_kernel = np.hstack(
            (0.5, 0.25, np.zeros(world_size_1d - 2)))
    elif x == world_size_1d -1:
        mm_kernel = np.hstack(
            (np.zeros(world_size_1d-3), 0.25, 0.75, 1.0))
    else:
        mm_kernel = np.hstack(
            (np.zeros(x-2), 0.25, 0.5, 0.25, 
                np.zeros(world_size_1d - (1+x))))
    
    if u == "back":
        mm_kernel = np.flip(mm_kernel)
        
    return mm_kernel


def discrete_bayes_filter(bel, d):
    if d["type"] == "perceptual":
        assert("This exercise does not use any perceptual data")
    else:
        # action
        bel_prime = np.zeros(np.shape(bel))
        for x in range(0,len(bel)):
            mm_kernel = motion_model(bel, d["u"], x)
            # Do convolution
            for x_prime in range(0, len(bel)):
                bel_prime[x] = np.sum(mm_kernel * bel)
        return bel_prime


bel = np.hstack ((np.zeros(9), 1, np.zeros(10)))

d_array = [
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "fwd"},
    {"type": "action", "u": "back"},
    {"type": "action", "u": "back"},
    {"type": "action", "u": "back"},
]

plt.figure()
plt.ion()
plt.show()

for d in d_array:
    plot_histogram(bel)
    bel = discrete_bayes_filter(bel, d)
    print("sum belief", np.sum(bel))

plt.ioff()
plt.show()
